In [ ]:
# example_srs_sim.py

import numpy as np
import tensorflow as tf
from sionna.nr import CarrierConfig
from sionna.nr.srs_config import SRSConfig
from sionna.nr.srs_pilot_pattern import SRSPilotPattern
from sionna.ofdm import ResourceGrid

def main():
    # 1) Create a CarrierConfig.
    carrier = CarrierConfig()
    carrier.n_size_grid = 24        # e.g., 24 resource blocks.
    carrier.subcarrier_spacing = 15 # in kHz.
    carrier.cyclic_prefix = "normal"
    carrier.slot_number = 10
    carrier.frame_number = 0

    # 2) Create SRSConfig objects for two transmitters.
    srs1 = SRSConfig(carrier_config=carrier)
    srs1.num_srs_ports = 2
    srs1.symbol_start = 10
    srs1.num_srs_symbols = 2
    srs1.c_srs = 5
    srs1.b_srs = 0
    srs1.k_tc = 2
    srs1.srs_period = [2, 1]  # e.g., period = 2 slots, offset = 1.
    srs1.n_srs_id = 123

    srs2 = SRSConfig(carrier_config=carrier)
    srs2.num_srs_ports = 1
    srs2.symbol_start = 11
    srs2.num_srs_symbols = 1
    srs2.c_srs = 3
    srs2.b_srs = 1
    srs2.srs_period = 'on'    # always on.
    srs2.n_srs_id = 45

    # 3) Build a pilot pattern from the SRS configurations.
    pilot_pattern = SRSPilotPattern([srs1, srs2])

    print("Pilot pattern created.")
    print("Mask shape:", pilot_pattern.mask.shape)     # Expected: [num_tx, max_ports, num_sym, num_sc]
    print("Pilots shape:", pilot_pattern.pilots.shape)   # Expected: [num_tx, max_ports, num_pilot_REs]

    # 4) Create a ResourceGrid for a single slot.
    num_tx = pilot_pattern.mask.shape[0]         # Number of transmitters.
    num_streams_per_tx = pilot_pattern.mask.shape[1]  # Using max_ports as streams per tx.
    rg = ResourceGrid(
        num_ofdm_symbols=carrier.num_symbols_per_slot,
        fft_size=carrier.n_size_grid * 12,
        num_tx=num_tx,
        num_streams_per_tx=num_streams_per_tx,
        pilot_pattern=pilot_pattern,
        cyclic_prefix_length=carrier.cyclic_prefix_length
    )

    # 5) Map the SRS pilot pattern into the resource grid.
    x_tx = rg.apply_pilot_pattern(None)
    print("Resource grid shape after applying pilot pattern:", x_tx.shape)

if __name__ == "__main__":
    main()
